## VQE ansatz: 001

In [1]:
import os
import numpy


import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer


import sys
sys.path.append("..")
from wordsToNumbers import Corpus
from wordsToNumbers import fibonacci_vocabulary

from wordsToQubits import put_word_on_sphere

from utils import get_corpus_from_directory, working_window, get_word_from_sphere

In [2]:
np.random.seed(73)

## Corpus

In [3]:
corpus_path='/Users/voicutu/Desktop/Qountry/CountryMixt'

corpus_tex = get_corpus_from_directory(corpus_path, limit=1)

corpus= Corpus(corpus_tex)
print(corpus.prop())

nr. words:1648 
nr. distinct words: 499 
len.text/len.vocab:3.302605210420842


In [4]:
parameterize_vovabulary = fibonacci_vocabulary(corpus.vocabulary)

In [5]:
print("corpus:",corpus.split_text)

corpus: ['same', 'old', 'dive', 'same', 'old', 'end', 'of', 'the', 'work', 'week', 'drink', 'bartender', 'knows', 'my', 'name', 'but', 'i', 'dont', 'mind', 'she', 'kicks', 'em', 'up', 'strong', 'serves', 'me', 'up', 'right', 'and', 'here', 'i', 'go', 'again', 'im', 'drinkin', 'one', 'im', 'drinkin', 'two', 'i', 'got', 'my', 'heartache', 'medication', 'a', 'strong', 'dedication', 'to', 'gettin', 'over', 'you', 'turnin', 'me', 'loose', 'on', 'that', 'hardwood', 'jukebox', 'lost', 'in', 'neon', 'time', 'my', 'heartache', 'medication', 'well', 'it', 'suits', 'me', 'fine', 'and', 'im', 'drinkin', 'enough', 'to', 'take', 'you', 'off', 'my', 'mind', 'i', 'got', 'my', 'heartache', 'medication', 'well', 'in', 'my', 'time', 'of', 'dying', 'dont', 'want', 'nobody', 'to', 'mourn', 'all', 'i', 'want', 'for', 'you', 'to', 'do', 'is', 'take', 'my', 'body', 'home', 'well', 'well', 'well', 'so', 'i', 'can', 'die', 'easy', 'well', 'well', 'well', 'well', 'well', 'well', 'so', 'i', 'can', 'die', 'easy', 

## Training set 

In [6]:
history_lenghth = 3

In [7]:
x,y = working_window(history_lenghth, splited_text=corpus.split_text)

In [8]:
print("len training set:", len(x))

len training set: 1644


## Circuit

In [9]:
dev = qml.device("default.qubit", wires=history_lenghth+1)

# circuit initializer
def circuit_initializer(words):
    for i in range(len(words)):
        put_word_on_sphere(words[i], qubit=i)

In [10]:
def layer_type1(param, wires=[0,1,2]):

    qml.CRZ( param[0], wires=[0,1])
    qml.CRX( param[1], wires=[0,1])
    qml.CRY( param[2], wires=[0,1])

    qml.CRZ( param[3], wires=[1,2])
    qml.CRX( param[4], wires=[1,2])
    qml.CRY( param[5], wires=[1,2])

    qml.CRZ( param[6], wires=[2,3])
    qml.CRX( param[7], wires=[2,3])
    qml.CRY( param[8], wires=[2,3])

    qml.RX( param[9], wires=0)
    qml.RY( param[10], wires=0)

    qml.RX( param[11], wires=1)
    qml.RY( param[12], wires=1)

    qml.RX( param[13], wires=2)
    qml.RY( param[14], wires=2)

    qml.RX( param[15], wires=3)
    qml.RY( param[16], wires=3)


In [11]:
@qml.qnode(dev)
def next_gen(params, x, obs='z'):
    """
    obs:'z', 'x' or 'y'
    """

    # initialize the circuit 
    circuit_initializer(x)

    # 
    for param in params:
        layer_type1(param, wires=[0,1,2])
    #circuit_initializer(x) # just for a test 


    # measure 
    if obs=='z':
        return  qml.expval(qml.PauliZ(3))
    if obs=='x':
        return qml.expval(qml.PauliX(3))
    if obs=='y':
        return qml.expval(qml.PauliY(3))



In [12]:
x_vec=[ parameterize_vovabulary[w] for w in x[0]]
print("x:",x[0])
print("x_vec:",x_vec)

x: ['same', 'old', 'dive']
x_vec: [[-0.5174781860030842, -0.6546184738955823, 0.5510816460793957], [-0.19127602086694068, 0.26104417670682734, 0.9461867794726179], [-0.3628495838337743, 0.9317269076305221, 0.015004969474187188]]


In [13]:
x_vec=[ parameterize_vovabulary[w] for w in x[0]]
print("x:",x[0])
print("x_vec:",x_vec)

params= np.random.uniform(size=(1, 17), requires_grad=True)

print("example tensor:", x_vec )
print("\n\n")
print(qml.draw(next_gen)(params,x_vec,obs='z'))

x: ['same', 'old', 'dive']
x_vec: [[-0.5174781860030842, -0.6546184738955823, 0.5510816460793957], [-0.19127602086694068, 0.26104417670682734, 0.9461867794726179], [-0.3628495838337743, 0.9317269076305221, 0.015004969474187188]]
example tensor: [[-0.5174781860030842, -0.6546184738955823, 0.5510816460793957], [-0.19127602086694068, 0.26104417670682734, 0.9461867794726179], [-0.3628495838337743, 0.9317269076305221, 0.015004969474187188]]



 0: ──RY(5.3)───RZ(0.902)───╭C──────────╭C──────────╭C───────────RX(0.311)───RY(0.404)────────────────────────────────────────────────────────────────────────┤     
 1: ──RY(5.95)──RZ(-0.938)──╰RZ(0.643)──╰RX(0.539)──╰RY(0.512)──╭C──────────╭C──────────╭C───────────RX(0.597)───RY(0.241)────────────────────────────────────┤     
 2: ──RY(4.73)──RZ(-1.2)────────────────────────────────────────╰RZ(0.61)───╰RX(0.493)──╰RY(0.214)──╭C──────────╭C──────────╭C──────────RX(0.906)──RY(0.159)──┤     
 3: ──────────────────────────────────────────────────────────

In [14]:
pred_vector=[ next_gen(params,x_vec, obs=o) for o in ['x', 'y', 'z']]

## Learning

In [15]:
def pred_target_distance(pred_vector, target):

    distance= np.linalg.norm(pred_vector-target)

    #np.sqrt((pred_vector[0]-target[0])*(pred_vector[0]-target[0])+
    #                 (pred_vector[1]-target[1])*(pred_vector[1]-target[1])+
    #                 (pred_vector[2]-target[2])*(pred_vector[2]-target[2]))

    return distance

In [16]:
print("target word:",y[0])
print("target vector:",parameterize_vovabulary[y[0]] )
print("prediction:",pred_vector)
pred_word= get_word_from_sphere(pred_vector, parameterize_vovabulary)
print("prediction word:",pred_word)
distance= pred_target_distance(np.array(pred_vector), np.array(parameterize_vovabulary[y[0]]))
print("distance:", distance)

target word: same
target vector: [-0.5174781860030842, -0.6546184738955823, 0.5510816460793957]
prediction: [tensor(0.41123432, requires_grad=True), tensor(-0.66660704, requires_grad=True), tensor(0.43325996, requires_grad=True)]
prediction word: then
distance: 0.9362331935842173


In [18]:
distance= pred_target_distance( np.array(parameterize_vovabulary[y[1]]), np.array(parameterize_vovabulary[y[1]]))
print("distance check",distance)

distance check 0.0


In [19]:
def cost( par , x, y):

    predictions = [[next_gen(par, w_input,obs='x') ,next_gen(par,w_input,obs='y') ,next_gen(par,w_input,obs='z') ] for w_input in x ]

    c=0.0
    for i in range(len(predictions)):
        c = c+ pred_target_distance(np.array(predictions[i]), y)

    c=c/len(predictions)

    return np.array(c)

In [20]:
def accuracy(predictions, y):
    pred_words=[ get_word_from_sphere(p_v, parameterize_vovabulary) for p_v in predictions]
    target_words=[ get_word_from_sphere(p_v, parameterize_vovabulary) for p_v in y]
    ac=0
    for i in range(len(pred_words)):
        if pred_words[i]==target_words[i]:
            ac=ac+1
    return ac/len(pred_words)

## Training:

In [21]:
## shuffling data 
index = np.array([ i for i in range(len(x))])
print(index)

[   0    1    2 ... 1641 1642 1643]


In [22]:
X_train= []
Y_train= []
Y_train_w= []

for i in range(len(x)):
    vec = [parameterize_vovabulary[w] for w in x[i]]
    X_train.append(vec)
    Y_train.append(parameterize_vovabulary[y[i]])
    Y_train_w.append(y[i])
    
X_train= np.array(X_train)
Y_train= np.array(Y_train)

In [23]:
def iterate_batches(X,Y, batch_size):
    X1 = [torch.reshape(x[0], (1, 2 ** (len(spec.latent_qubits) + len(spec.trash_qubits)))) for x in X]
    X2 = []
    for i in range(len(X1)):
        X2.append([X1[1], X[i][1]])
    X = X2
    random.shuffle(X)

    batch_list = []
    batch = []
    for x in X:
        if len(batch) < batch_size:
            batch.append(x)

        else:
            batch_list.append(batch)
            batch = []
    if len(batch) != 0:
        batch_list.append(batch)
    return batch_list

In [24]:
## Model parameters 

num_layers= 1
layer_param= 17
params = np.random.uniform(size=(2, layer_param), requires_grad=True)

learning_rate= 0.6
opt = AdamOptimizer(learning_rate, beta1=0.9, beta2=0.999)

nr_epochs= 300

In [ ]:
ls_progres=[]
ac_progres=[]
for e in range(nr_epochs):
    params, ls = opt.step_and_cost(lambda p: cost( par=p, x=X_train, y=Y_train),params)

    print("Iter:{} | train_cost:{}".format(e, ls))
    ls_progres.append(ls)


    if e%10==0:
        predictions = [[next_gen(params, w_input,obs='x') ,next_gen(params,w_input,obs='y') ,next_gen(params,w_input,obs='z') ] for w_input in X_train ]
        ac=accuracy(predictions, y=Y_train)

        print("ac:",ac)
        ac_progres.append(ac)
  

Iter:0 | train_cost:53.60082886916693
ac: 0.0036496350364963502
Iter:1 | train_cost:51.22300344453689
Iter:2 | train_cost:43.94168400175275
Iter:3 | train_cost:43.02241166220944
Iter:4 | train_cost:42.20706998707202
Iter:5 | train_cost:42.22409169057897
Iter:6 | train_cost:41.569128379960745
Iter:7 | train_cost:41.62247286223286
Iter:8 | train_cost:41.42697478304665
Iter:9 | train_cost:41.5571161180561
Iter:10 | train_cost:41.756217205647744
ac: 0.0006082725060827251
Iter:11 | train_cost:41.81138947351213
Iter:12 | train_cost:41.50899039197157
Iter:13 | train_cost:41.284097932138984
Iter:14 | train_cost:41.23511784896323
Iter:15 | train_cost:41.445282858198446
Iter:16 | train_cost:41.17631706803689
Iter:17 | train_cost:41.183101421089425
Iter:18 | train_cost:41.23648509725077
Iter:19 | train_cost:41.10445623032547
Iter:20 | train_cost:41.017359535920136
ac: 0.0024330900243309003
Iter:21 | train_cost:41.03232950789743
Iter:22 | train_cost:40.9921434744548
Iter:23 | train_cost:40.9116885

## Results

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure()
plt.plot([x for x in range(0,len(ls_progres))],np.array(ls_progres),label="train loss")


plt.legend()
plt.title("loss_VQE_001_C1",)
plt.xlabel("epoch")
plt.ylabel("loss")

print("last loss:",ls_progres[-1])

In [ ]:
fig = plt.figure()
plt.plot([x for x in range(0,len(ac_progres)*10,10)],np.array(ac_progres),label="train accuracy")


plt.legend()
plt.title("accuracy_VQE_001_C1",)
plt.xlabel("epoch")
plt.ylabel("accyracy")

print("accuracy loss:",ac_progres[-1])

In [ ]:
for i in range(len(x)):
    
    words_vec = [parameterize_vovabulary[w] for w in x[i]]
    pred_vector=[ next_gen(params,words_vec, obs=o) for o in ['x', 'y', 'z']]
    pred_word= get_word_from_sphere(pred_vector, parameterize_vovabulary)
    
    text=""
    for w in x[i];
    text=text+w+" "
    
    print("{} {}|{}".format(text, pred_word,y[i]))